In [24]:
from pyspark.sql import SparkSession, functions as F
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Process Cleaned JSON Data").getOrCreate()

df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

filtered_players = (
    df.filter(
        (df.primaryPosition == "Cf")  #|
        # (df.primaryPosition == "Cm") |
        # (df.primaryPosition == "Lm") |
        # (df.primaryPosition == "Rm")  |
        # (df.primaryPosition == "Am")
    )
    .sort(df.rTalent.desc(), df.power.desc())
    .select(
        "id",
        "name",
        "country_name",
        "primaryPosition",
        "secondaryPosition",
        "club",
        "rTalent",
        "power",
        "team_id",
    )
)

# Виводимо результати
filtered_players.show(59, truncate=False)

# Перетворюємо Spark DataFrame у Pandas DataFrame
pandas_df = filtered_players.toPandas()

# Зберігаємо у файл Excel
output_path = "/home/yevhen/Проекти/butsa/json/filtered_players.xlsx"
# with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
#     pandas_df.to_excel(writer, sheet_name="Forwards", index=False)

wb = load_workbook(output_path)
ws = wb["All Players"]  # Відкриваємо потрібний лист

# Форматування клітинок: заповнення кольором, якщо rTalent > 4.5
fill_highlight = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):  # Починаємо з другого рядка
    club_cell = row[6 - 1]  # Індекс колонки (мінус 1, бо індекси Python починаються з 0)
    if club_cell.value == "Ницца":
        for cell in row:
            cell.fill = fill_highlight

# Зберігаємо файл
wb.save(output_path)

+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|id    |name                 |country_name       |primaryPosition|secondaryPosition|club                 |rTalent|power|team_id                          |
+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|851766|Альваро Мората       |Испания            |Cf             |                 |Сирама Де Намикиа    |5.1    |22.86|https://www.butsa.ru/roster/9085 |
|852992|Зорепад Калапач      |Украина            |Cf             |Am               |Академия Пушкаша     |5.0    |31.21|https://www.butsa.ru/roster/9004 |
|851892|Диего Коста          |Испания            |Cf             |                 |Виборг               |5.0    |21.0 |https://www.butsa.ru/roster/11398|
|852336|Михаил Гуськов       |Россия             |Cf             |    

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Average rTalent by Club").getOrCreate()

# Завантаження JSON у Spark DataFrame
df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

# Обчислення середнього rTalent по Club
avg_rTalent_by_club = (
    df.groupBy("club")  # Групуємо за країною та клубом
    .agg(
        F.avg("rTalent").alias("avg_rTalent"),  # Обчислюємо середнє значення rTalent
        F.avg("power").alias("avg_power"),  # Обчислюємо середнє значення power
        F.count("id").alias("count"),  # Рахуємо кількість гравців у клубі
    )
    .filter(F.col("count") >= 6)  # Фільтруємо клуби з середнім rTalent > 0
    .select("club", "avg_rTalent", "avg_power", "count")
    .orderBy(F.desc("avg_rTalent"))  # Сортуємо за спаданням середнього rTalent
)

# Виводимо результати
avg_rTalent_by_club.show(40, truncate=False)

+--------------+------------------+------------------+-----+
|club          |avg_rTalent       |avg_power         |count|
+--------------+------------------+------------------+-----+
|Кильмес       |4.8999999999999995|31.27285714285714 |7    |
|Спартак Москва|4.47              |29.0              |10   |
|Аль Барк Тарим|4.45              |30.496666666666666|12   |
|Уракан Манати |4.445454545454545 |27.576363636363634|11   |
|ПФК Рось      |4.357142857142857 |26.69             |7    |
|Газ Метан     |4.35              |36.425            |8    |
|Виллем II     |4.328571428571428 |21.765000000000004|14   |
|АнтА          |4.277777777777778 |24.28333333333333 |9    |
|Ницца         |4.20909090909091  |20.074545454545458|11   |
|Х.Б.С.        |4.166666666666667 |28.231666666666673|6    |
+--------------+------------------+------------------+-----+



In [25]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


def upload_excel_to_google_drive(
    file_path,
    file_name,
    mime_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
):
    """
    Завантажує Excel-файл у Google Диск як Google Sheet.
    Якщо файл із такою назвою вже існує, він буде перезаписаний.

    :param file_path: Шлях до локального файлу Excel.
    :param file_name: Назва файлу, який буде створено або перезаписано на Google Диску.
    :param mime_type: Тип файлу для завантаження (за замовчуванням Google Sheet).
    """
    # Авторизація через GoogleAuth
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(
        "/home/yevhen/Проекти/butsa/json/client_secret_935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com.json"
    )
    gauth.LocalWebserverAuth()  # Відкриває браузер для авторизації
    drive = GoogleDrive(gauth)

    # Пошук файлу з такою ж назвою
    file_list = drive.ListFile(
        {"q": f"title='{file_name}' and trashed=false"}
    ).GetList()

    if file_list:
        # Якщо файл знайдено, оновлюємо його
        file = file_list[0]  # Беремо перший файл із такою назвою
        file.SetContentFile(file_path)
        file.Upload()
        print(f"Файл '{file_name}' успішно оновлено на Google Диску.")
    else:
        # Якщо файл не знайдено, створюємо новий
        file = drive.CreateFile({"title": file_name, "mimeType": mime_type})
        file.SetContentFile(file_path)
        file.Upload()


upload_excel_to_google_drive(
    file_path="/home/yevhen/Проекти/butsa/json/filtered_players.xlsx",
    file_name="Filtered players",
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



KeyboardInterrupt: 